In [1]:
from subtitle_process import fetch_subtitle
import getpass
import os
import json
import re
from templates import *
from functions import inference

In [2]:
# 设置临时环境变量
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("bilibili_ssid")

In [3]:
with open("url.json", 'r') as file:
    data = json.load(file)

In [4]:
data[0]

'BV1iT411q7GM'

In [5]:
problems = []
for bv in data:
    problems.append(fetch_subtitle(bv))

Extracted segment: 1iT411q7GM
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/484220007107407896653ecb5f2122578ac2f3848cad97b7709?auth_key=1720583898-dba776369c2f4465893916653e77b918-0-6a2976ec152dcf06b8c11bc2e9d7c899
Extracted segment: 15Y4y1S7Kb
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/654142091107406688224ec815f898b393f51d4bf087e552219?auth_key=1720583901-5953bb5135d646eab8b9b3ff2ca3ab11-0-ad33914df45c0ff62d62d7ef8a472918
Extracted segment: 1y24y1L7dZ
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/7818091071075718488fcd05d7f2bf5e7db1e705d9f8d70fa22?auth_key=1720583905-190e017888c64f26acce37c6112cdaed-0-b3ce7c89784c2c5c94c2aa275c9ad2b0
Extracted segment: 1Y84y1g7ah
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/6117549431075727729a9c113ae3840f

In [6]:
def extract_question_explanations(responses):
    questions_and_explanations = []
    for response in responses:
        pattern = re.compile(r"### 原题：\n(.+?)\n\n### 讲解：\n(.+?)\n\n### 相似题一：\n(.+?)\n\n### 相似题一讲解：\n(.+?)\n\n", re.DOTALL)
        matches = pattern.findall(response)

        for match in matches:
            questions_and_explanations.append({
                "original_question": match[0].strip(),
                "original_explanation": match[1].strip(),
                "similar_question_1": match[2].strip(),
                "similar_explanation_1": match[3].strip(),
            })

    with open('casestudy_problem_answer.json', 'w', encoding='utf-8') as f:
        json.dump(questions_and_explanations, f, ensure_ascii=False, indent=4)


In [7]:
template_sub_1_similar = r"""
以下是一段视频字幕，请提取出题目和老师对题目的讲解，同时构造一个相似题目并生成与老师讲解风格相似的讲解。请按如下格式输出：

### 原题：
（提取出的原题）

### 讲解：
（提取出的讲解）

### 相似题一：
（构造的相似题目一）

### 相似题一讲解：
（构造的相似题目一的讲解）

视频字幕：
{subtitle}

注意事项：
1. 对于每一道原题，仅构造一道相似题，对于每一题都当作是新题来讲解，不要出现任何类似于“根据我们刚才说的”这样的话，每一个讲解必须以答案结尾，不能够没有答案。
2. 保留老师讲解风格但是把和讲解无关的内容过滤掉，比如自问自答的部分都要删除掉。
"""

template_expl = r"""
    Role:
        你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一的讲解一个题目。 
    Goals:
        根据给定的板书内容像老师一样讲解板书内容，根据输入决定你正在输出的是讲解几，比如如果输入是板书一，那么输出就是讲解一，以此类推。

    严格按照以下列字符串格式输出:
        ### 讲解一:    ### 讲解二:      ...

    原题和参考信息:
        题目:{question}
        参考信息:{ref}

    注意事项:
        1. 你的讲解要尽量贴合人类老师的讲解风格，言简意赅，清晰明了。
        2. 讲解内容应针对板书的具体步骤进行详细说明，解释每一步的思路和方法。
        3. 确保每一个讲解都对应一个板书。
        4. 板书和讲解的编号必须一一对应，格式必须严格遵守。
        5. 讲解内容应突出解题思路和方法，避免过多冗余信息。
        6. 你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇
        7. 其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题，别用括号1234。
            d: 初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
            e: 对于所有题目尽可能有最简单的方式来进行解答，以最大限度上避免超纲的情况发生。

    下面是板书的内容(请确保每一个讲解都对应一个板书)：
    {notes}

    下面是一些名师讲解的其他题目，请你学习他们的讲解风格，并运用到板书的讲解中(注意：你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇)。
    注意：初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
    {model_expl}
"""

In [8]:
raw_problem_answer = []
for problem in problems:
    raw_problem_answer.append(await inference({"subtitle": problem}, template_sub_1_similar))


executing
### 原题：
两个数相减的差是7.2。如果被减数增加8.6，减数减少2.7，则现在的差是多少？

### 讲解：
首先我们要理解题目中的关键变化。被减数增加8.6，那么差也会增加8.6。减数减少2.7，那么差也会增加2.7。最初的差是7.2，所以新的差是7.2 + 8.6 + 2.7。我们来计算一下，7.2 + 8.6 = 15.8，再加上2.7，15.8 + 2.7 = 18.5。所以现在的差是18.5。

### 相似题一：
两个数相减的差是5.4。如果被减数增加6.3，减数减少1.8，则现在的差是多少？

### 相似题一讲解：
首先我们要理解题目中的关键变化。被减数增加6.3，那么差也会增加6.3。减数减少1.8，那么差也会增加1.8。最初的差是5.4，所以新的差是5.4 + 6.3 + 1.8。我们来计算一下，5.4 + 6.3 = 11.7，再加上1.8，11.7 + 1.8 = 13.5。所以现在的差是13.5。executing
### 原题：
一个九位数各个数位上的数字之和是13，其中万位上的数字是一位上的两倍，问这个数最大是多少？

### 讲解：
首先，我们把这个九位数写成九个方框：□□□□□ □□□□。这样可以帮助我们快速找到对应的数位。题目中提到万位上的数字是一位上的两倍，我们先找到万位和个位的位置。

为了使这个数最大，我们需要尽量让高位的数字大。假设万位上的数字是9，那么个位上的数字应该是9的一半，即18，这显然不可能。所以我们从8开始尝试，8的两倍是16，也不行。继续尝试7，7的两倍是14，还是不行。直到我们尝试4，4的两倍是8，这才符合条件。

现在我们知道万位是4，个位是8。接下来，我们需要确保所有数位上的数字之和是13。4和8加起来是12，还差1。为了使这个数最大，我们把1填在尽量靠前的位置，其余位置填0。

所以，这个九位数最大是400000008。

### 相似题一：
一个八位数各个数位上的数字之和是15，其中千位上的数字是个位上的三倍，问这个数最大是多少？

### 相似题一讲解：
首先，我们把这个八位数写成八个方框：□□□□ □□□□。这样可以帮助我们快速找到对应的数位。题目中提到千位上的数字是个位上的三倍，我们先找到千位和个位的位置。

为了使这个数最大，我们需要尽量让高位的数字大。假设千位上的

In [9]:
responses = raw_problem_answer
questions_and_explanations = []
for response in responses:
    # print(type(response))
    pattern = re.compile(r"### 原题：\n(.*?)\n\n### 讲解：\n(.*?)\n\n### 相似题一：\n(.*?)\n\n### 相似题一讲解：\n(.*?)\n\n", re.DOTALL)
    matches = pattern.findall(response)
    # print(matches)
    for match in matches:
        questions_and_explanations.append({
            "original_question": match[0].strip(),
            "original_explanation": match[1].strip(),
            "similar_question_1": match[2].strip(),
            "similar_explanation_1": match[3].strip(),
        })

with open('casestudy_problem_answer.json', 'w', encoding='utf-8') as f:
    json.dump(questions_and_explanations, f, ensure_ascii=False, indent=4)


In [12]:
extract_question_explanations(raw_problem_answer)

In [10]:
with open("casestudy_problem_answer.json", 'r') as file:
    data = json.load(file)

data

[{'original_question': '一个九位数各个数位上的数字之和是13，其中万位上的数字是一位上的两倍，问这个数最大是多少？',
  'original_explanation': '首先，我们把这个九位数写成九个方框：□□□□□ □□□□。这样可以帮助我们快速找到对应的数位。题目中提到万位上的数字是一位上的两倍，我们先找到万位和个位的位置。\n\n为了使这个数最大，我们需要尽量让高位的数字大。假设万位上的数字是9，那么个位上的数字应该是9的一半，即18，这显然不可能。所以我们从8开始尝试，8的两倍是16，也不行。继续尝试7，7的两倍是14，还是不行。直到我们尝试4，4的两倍是8，这才符合条件。\n\n现在我们知道万位是4，个位是8。接下来，我们需要确保所有数位上的数字之和是13。4和8加起来是12，还差1。为了使这个数最大，我们把1填在尽量靠前的位置，其余位置填0。\n\n所以，这个九位数最大是400000008。',
  'similar_question_1': '一个八位数各个数位上的数字之和是15，其中千位上的数字是个位上的三倍，问这个数最大是多少？',
  'similar_explanation_1': '首先，我们把这个八位数写成八个方框：□□□□ □□□□。这样可以帮助我们快速找到对应的数位。题目中提到千位上的数字是个位上的三倍，我们先找到千位和个位的位置。'},
 {'original_question': '6161 x 39 - 3939 x 61',
  'original_explanation': '这道题目中有两个乘号和一个减号，满足乘法分配率的逆运算形式，但没有相同的数。我们需要将其转化为有相同数的形式。首先，我们将6161拆分为61 x 101。具体步骤如下：\n\n6161 = 61 x 100 + 61 = 61 x (100 + 1) = 61 x 101\n\n同样地，我们将3939拆分为39 x 101：\n\n3939 = 39 x 100 + 39 = 39 x (100 + 1) = 39 x 101\n\n将这些等量代换到原题中：\n\n6161 x 39 - 3939 x 61 = (61 x 101) x 39 - (39 x 101) x 61\n\n观察减号两边的数字，发现它们相同：\n\n

In [11]:
model_explanation = ""
for ps in data[:3]:
    model_explanation += "题目:\n" + ps['original_question'] + '\n\n'
    model_explanation += "讲解:\n" + ps['original_explanation'] + '\n\n'
    # model_explanation += "题目:\n" + ps['similar_question_1'] + '\n\n'
    # model_explanation += "讲解:\n" + ps['similar_explanation_1'] + '\n\n'
model_explanation

'题目:\n一个九位数各个数位上的数字之和是13，其中万位上的数字是一位上的两倍，问这个数最大是多少？\n\n讲解:\n首先，我们把这个九位数写成九个方框：□□□□□ □□□□。这样可以帮助我们快速找到对应的数位。题目中提到万位上的数字是一位上的两倍，我们先找到万位和个位的位置。\n\n为了使这个数最大，我们需要尽量让高位的数字大。假设万位上的数字是9，那么个位上的数字应该是9的一半，即18，这显然不可能。所以我们从8开始尝试，8的两倍是16，也不行。继续尝试7，7的两倍是14，还是不行。直到我们尝试4，4的两倍是8，这才符合条件。\n\n现在我们知道万位是4，个位是8。接下来，我们需要确保所有数位上的数字之和是13。4和8加起来是12，还差1。为了使这个数最大，我们把1填在尽量靠前的位置，其余位置填0。\n\n所以，这个九位数最大是400000008。\n\n题目:\n6161 x 39 - 3939 x 61\n\n讲解:\n这道题目中有两个乘号和一个减号，满足乘法分配率的逆运算形式，但没有相同的数。我们需要将其转化为有相同数的形式。首先，我们将6161拆分为61 x 101。具体步骤如下：\n\n6161 = 61 x 100 + 61 = 61 x (100 + 1) = 61 x 101\n\n同样地，我们将3939拆分为39 x 101：\n\n3939 = 39 x 100 + 39 = 39 x (100 + 1) = 39 x 101\n\n将这些等量代换到原题中：\n\n6161 x 39 - 3939 x 61 = (61 x 101) x 39 - (39 x 101) x 61\n\n观察减号两边的数字，发现它们相同：\n\n(61 x 101) x 39 - (39 x 101) x 61 = 61 x 101 x 39 - 39 x 101 x 61\n\n由于两边的乘法结果相同，最终结果为零：\n\n答案：0\n\n题目:\n一个等腰三角形其中一个角的度数为50度，求另外两个角的度数。\n\n讲解:\n首先，等腰三角形有一个顶角和两个底角。题目中给出的50度可以是顶角也可以是底角，所以这道题有两个答案。\n\n1. 当50度是顶角时：\n   - 等腰三角形的内角和为180度。\n   - 另外两个底角的度数为：(180度 - 50度

In [15]:
orig_answers = []
orig_notes = []
orig_expls = []
refined_explanation = []
questions = []

In [16]:
'''
对于每一个<原题,相似题>，用两个prompt来分别对相似题进行解答
    1. 用original prompt对相似题直接进行解答
    2. 将原题解答及相似题解答作为few shots examples喂给另一个prompt来对比两种的效果哪一个比较好
'''

with open("casestudy_problem_answer.json", 'r') as file:
    problem_set = json.load(file)


for ps in problem_set:
    orig_answers.append(await inference({"question": ps['similar_question_1'], "ref_form": ref_form, "ref": ref}, template_orig))
    pattern1 = r"### 板书一:[\s\S]*?(?:### 板书二:[\s\S]*?)?(?=### 讲解一)"
    orig_note = re.search(pattern1, orig_answers[-1]).group()
    pattern2 = r"### 讲解一:[\s\S]*"
    orig_expl = re.search(pattern2, orig_answers[-1]).group()
    orig_notes.append(orig_note)
    orig_expls.append(orig_expl)
    questions.append(ps['similar_question_1'])
    refined_explanation.append(await inference({"question": ps['similar_question_1'], "ref": ref, "notes": orig_note, "model_expl": model_explanation}, template_expl))

# print(orig_notes)
# print(refined_explanation)    

executing
知识点讲解:本题考查的知识点是数位和问题，解题关键点是合理分配各个数位上的数字使得总和最大。

### 板书一:
解：设这个八位数为$ABCDEFGH$，其中$A, B, C, D, E, F, G, H$分别表示各个数位上的数字。
由题意得：
$A + B + C + D + E + F + G + H = 15$
$D = 3H$

### 板书二:
$\because$ $D = 3H$
$\therefore$ $A + B + C + 3H + E + F + G + H = 15$
$A + B + C + E + F + G + 4H = 15$

### 板书三:
为了使这个数最大，尽量让高位上的数字大。
设$H = 1$，则$D = 3 \times 1 = 3$
$A + B + C + E + F + G + 4 \times 1 = 15$
$A + B + C + E + F + G + 4 = 15$
$A + B + C + E + F + G = 11$

### 板书四:
尽量让$A$最大，设$A = 9$
$9 + B + C + E + F + G = 11$
$B + C + E + F + G = 2$

### 板书五:
尽量让$B$最大，设$B = 2$
$2 + C + E + F + G = 2$
$C + E + F + G = 0$

### 板书六:
$\because$ $C, E, F, G$都是非负整数
$\therefore$ $C = 0, E = 0, F = 0, G = 0$

### 板书七:
$\therefore$ 这个八位数为$92030001$

故答案是$92030001$

### 讲解一:
首先设定八位数的各个数位上的数字为$A, B, C, D, E, F, G, H$，并根据题意列出总和等于15的方程，同时千位上的数字是个位上的三倍。

### 讲解二:
将千位上的数字用个位上的数字表示，简化方程。

### 讲解三:
为了使这个数最大，尽量让高位上的数字大。设定个位上的数字为1，计算出千位上的数字为3，并代入总和方程。

### 讲解四:
继续让最高位的数字尽量大，设定最高位的数字为9，并计算剩余数位的和。

### 讲解五:
继续让次高位的数字尽量大，

In [17]:
with open("compare_results_p4.md", 'a') as file:
    for i in range(len(questions)):
        file.write("# 原题：\n" + questions[i] + '\n\n')
        file.write("# 原本板书：\n" + orig_notes[i] + '\n\n')
        file.write("# 原本解答：\n" + orig_expls[i] + '\n\n')
        file.write("# 新解答：\n" + refined_explanation[i] + '\n\n')

In [31]:
print(len(orig_expls))

12


In [32]:
import re

text = """
知识点讲解:本题考查的知识点是因式分解与方程求解，解题关键点是利用平方差公式将原式化简并求解。

###板书一:
解：
$(2a+2b+1)(2a+2b-1)=63$

$设x=2a+2b$

$则(x+1)(x-1)=63$

$x^2-1=63$

$x^2=64$

$x=±8$

$2a+2b=8$

$a+b=4$

$2a+2b=-8$

$a+b=-4$

故答案是$a+b=4$或$a+b=-4$

###讲解一:
首先，我们将原式$(2a+2b+1)(2a+2b-1)=63$进行因式分解，利用平方差公式$(x+1)(x-1)=x^2-1$，设$x=2a+2b$，则原式变为$x^2-1=63$。

###讲解二:
接下来，我们解方程$x^2-1=63$，得到$x^2=64$，所以$x=±8$。然后将$x$代入$2a+2b=x$，分别得到$2a+2b=8$和$2a+2b=-8$，从而求得$a+b=4$或$a+b=-4$。故答案是$a+b=4$或$a+b=-4$。
"""

pattern = r"###讲解一:[\s\S]*"
match = re.search(pattern, text).group()

if match:
    print(match)
else:
    print("No match found")


###讲解一:
首先，我们将原式$(2a+2b+1)(2a+2b-1)=63$进行因式分解，利用平方差公式$(x+1)(x-1)=x^2-1$，设$x=2a+2b$，则原式变为$x^2-1=63$。

###讲解二:
接下来，我们解方程$x^2-1=63$，得到$x^2=64$，所以$x=±8$。然后将$x$代入$2a+2b=x$，分别得到$2a+2b=8$和$2a+2b=-8$，从而求得$a+b=4$或$a+b=-4$。故答案是$a+b=4$或$a+b=-4$。

